In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('../input/thoraric-surgery/ThoraricSurgery.csv', index_col = 'id')

описание датасета [тут](https://archive.ics.uci.edu/ml/datasets/Thoracic+Surgery+Data)

In [ ]:
data.head(2)

In [ ]:
data.info()

очень маленький датасет.

Преобразуем T / F ячейки в 1 / 0

In [ ]:
data[['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19', 'PRE25', 'PRE30', 'PRE32', 'Risk1Yr']] = \
(data[['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19', 'PRE25', 'PRE30', 'PRE32', 'Risk1Yr']] == 'T').astype(int)

In [ ]:
data.head(2)

т.к. данные в остальных ячейках уже содержат в себе цифровой код, можно преобразовать их в числа таким образом:

In [ ]:
data['DGN']   = data['DGN'].str[-1:].astype(int)
data['PRE6']  = data['PRE6'].str[-1:].astype(int)
data['PRE14'] = data['PRE14'].str[-1:].astype(int)

In [ ]:
data.describe(include='all')

переименуем для понятности

Attribute Information:

1. DGN: Diagnosis - specific combination of ICD-10 codes for primary and secondary as well multiple tumours if any (DGN3,DGN2,DGN4,DGN6,DGN5,DGN8,DGN1)
2. PRE4: Forced vital capacity - FVC (numeric)
3. PRE5: Volume that has been exhaled at the end of the first second of forced expiration - FEV1 (numeric)
4. PRE6: Performance status - Zubrod scale (PRZ2,PRZ1,PRZ0)
5. PRE7: Pain before surgery (T,F)
6. PRE8: Haemoptysis before surgery (T,F)
7. PRE9: Dyspnoea before surgery (T,F)
8. PRE10: Cough before surgery (T,F)
9. PRE11: Weakness before surgery (T,F)
10. PRE14: T in clinical TNM - size of the original tumour, from OC11 (smallest) to OC14 (largest) (OC11,OC14,OC12,OC13)
11. PRE17: Type 2 DM - diabetes mellitus (T,F)
12. PRE19: MI up to 6 months (T,F)
13. PRE25: PAD - peripheral arterial diseases (T,F)
14. PRE30: Smoking (T,F)
15. PRE32: Asthma (T,F)
16. AGE: Age at surgery (numeric)
17. Risk1Y: 1 year survival period - (T)rue value if died (T,F)

In [ ]:
col = ['Diagnosis','Forced_Capacity','Forced_Expiration','Zubrod_scale','Pain',' Haemoptysis','Dyspnoea',
       'Cough','Weakness','Size_of_tumor','diabetes','MI_6months','PAD','Smoker','Asthmatic','Age','Risk_1y']
data.columns = col

In [ ]:
data.head()

немного посмотрим на данные

In [ ]:
sns.pairplot(data[['Forced_Expiration','Smoker', 'Age', 'Risk_1y']], 
             hue='Risk_1y', diag_kws={'bw':1.5}, markers=['o', 'D'], height=2.5)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
mask=np.zeros_like(data.corr())
mask[np.triu_indices_from(mask)] = True
sns.heatmap(data.corr(), annot=True, linewidths=.1, cmap="YlGnBu", square=True, mask=mask, cbar=False)

есть пара ярковыраженных связей

In [ ]:
fig, ax = plt.subplots(figsize = (12,6))
sns.scatterplot(x='Diagnosis', y='Size_of_tumor', #hue='Risk_1y', 
                size='dgn_cnt', sizes=(20, 250),
                data=data.groupby(['Diagnosis','Size_of_tumor']).size().reset_index().rename(columns={0:'dgn_cnt'}))

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.barplot(x='Diagnosis', y='Risk_1y', 
            data = data, palette="Blues_d",
            ax=ax, ci=None)

т.е. нет смертей для DGN6, DGN1

In [ ]:
# но и наблюдений таких совсем мало..
data[data.Diagnosis.isin([1, 6])]

Предсказывать будем Диагноз, посмотрим на распределение

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.distplot(data.Diagnosis, kde=False)

данных по части диагнозов очень очень мало   
Объединим диагнозы 1,5,6,7,8 в один - всё равно точность по ним будет минимальная

In [ ]:
data1 = data.copy()
data1['Diagnosis'] = np.where(data['Diagnosis'].isin([1,5,6,7,8]), 0, data['Diagnosis'])
data1.head()

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.distplot(data1.Diagnosis, kde=False)

In [ ]:
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import learning_curve, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X = data1.drop(columns='Diagnosis')
y = data1.Diagnosis

пусть базовое предсказание будет без аугментации - обычная логистическая регрессия

In [ ]:
clf = LogisticRegression(class_weight = 'balanced')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, predictions))

там где было больше наблюдений - хоть какая-то точность вышла, остальные классы - по нулям

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
augm = ADASYN()
X_train_augm, y_train_augm = augm.fit_resample(np.array(X_train), np.array(y_train))

In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
pd.Series(y_train_augm).value_counts()

In [ ]:
clf = LogisticRegression()
clf.fit(X_train_augm, y_train_augm)
predictions = clf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, predictions))

чуть подтянулся 2й класс - который изначально был на 2м месте по кол-ву наблюдений.

Вероятно в такой ситуации остается только накапливать наблюдения до повышения точности, а алгоритм настроить по необходимости на (как пример) высокий recall по одному из классов, чтобы не пропустить критичный диагноз